# Part 2 Algorithm Approach


### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
It is clearly a classification problem as we have class labels passed=yes and passed=no. A regression problem would be something like the final grade of the students (even if you could still formulate that as a multiclass classification problem if you really want to)

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.keys())-1

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed']=='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed']=='no'])

# TODO: Calculate graduation rate
grad_rate = n_passed/n_students*100

# Print the results
print("Total number of students: {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed: {}".format(n_passed))
print("Number of students who failed: {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0   ICHA   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1   ICHA   F   17       U     GT3       T     1     1  at_home     other  ...   
2   ICHA   F   15       U     LE3       T     1     1  at_home     other  ...   
3   ICHA   F   15       U     GT3       T     4     2   health  services  ...   
4   ICHA   F   16       U     GT3       T     3     3    other     other  ...   

  higher internet  romantic  famrel  freetime goout Dalc Walc health absences  
0    yes       no        no       4         3     4    1    1     

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

#yb_all = y_all.copy()
#yb_all = yb_all.replace(['yes', 'no'], [1, 0])

Processed feature columns (48 total features):
['school_ICHA', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split


# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all, stratify=y_all, test_size=95, random_state=0)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, we will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. we will first discuss the reasoning behind choosing these three models by considering what we know about the data and each model's strengths and weaknesses. We will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
# 1. Decision Trees
## 1.1 General applications
Decision trees are used for supervised learning and can be both applied for regression and classification problems.

## 1.2 Strenght and weakness
### 1.2.1 Strenghts
- Easy to use
- Graphically intuitive (easy to understand how it works as the decision process itself is also visualizable) (white box approach)
- Can be used for both regression and classification
- Fast computing of the prediction result. Is in O(log(n)) where n is number of data points used for training.

### 1.2.2 Weaknesses
- Prone to overfitting (high variance)
    - Pruning would be a counter-measure
    - Alternative: Use decision tree in an ensemble learner
- If you have missing attribute values further work for completing them is needed (e.g. http://people.eecs.ku.edu/~jerzy/b24-miss.pdf )
- Unstable: If you change the data a little bit you can possibly end up with a total different tree (using them within an ensemble method can solve this problem)
- No guarantee that we have the global optimal decision tree as it uses greedy algorithms inside (e.g. maximizing information gain)

## 1.3 Why good for our specific problem
We don't have a lot of features so it is not super likely that our decision tree will completely overfit. There is also no single attribute value missing which makes it easy for us to just use decision trees. We don't need any preprocsessing. Also we have an offline problem where the data is not changing so we don't have the problem of unstable decision trees as we don't have data changes.

# 2. Support Vector Machines (SVMs)
## 2.1 General applications
Support Vector Machines are a supervised learning method for regression and classification problems. They create a decision boundary between the training sets to classify them.

## 2.2 Strenght and weakness
### 2.2.1 Strenghts
- Effective in high dimensional spaces
- Performs really good were we have a clear decison boundary
- Memory efficient as it only uses the support vecotors for the decision and not all the training points
- There is ONE best (optimal) line for the problem which is found by the SVM

### 2.2.2 Weaknesses
- Works bad with noisy data
- Huge computing time on large datasets

## 2.3 Why good for our specific problem
We don't have a big data set with our 395 students and 30 features which means that SVMs can calculate that in a reasonable amount of time. We also don't have any noise in our data set which also favors SVMs. 

# 3. Neural Networks
## 3.1 General applications
Artificial neural networks can learn (in theory) any arbitary function if they have enough data. They are used in a wide range of applications like image classification, speech recognition or translation.

## 3.2 Strenght and weakness
### 3.2.1 Strenghts
- Easy to implement on your own (at least the basic algorithm with back propagation)
- Good for big datasets
- Backprogagation convergence is guaranteed (slow though) 
    - Speedup of backpropagation possible with dropout, momentum based gradient update, batch normalization and other tricks

### 3.2.2 Weaknesses
- Black box which makes it hard to understand what is going on in the training process which again makes it harder to tune or adapt your network

## 3.3 Why good for our specific problem
not applicable as neural networks are only in the dev branch of sklearn available. Therefore I switched to a naive bayes classifier:

# 4. Naive Bayes Classifier
## 4.1 General applications
Used for spam filtering and document classification most of the time.

## 4.2 Strenght and weakness
### 4.2.1 Strenghts
- Extremly fast
- To get the parameters right they only need a small amount of training data 

### 4.2.2 Weaknesses
- Is a bad estimator which means that the probabilities of how confident the algorithm is are not that accurate in general

## 4.3 Why good for our specific problem
We don't have that much training data and we want to have a fast (minimum computing time) algorithm. Naive Bayes seems to promise to work with small data and seems to be quite fast.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [24]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for Student dropout: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}. \n".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [25]:
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import BaggingClassifier as BC
from sklearn.ensemble import RandomForestClassifier as RFC

# TODO: Initialize the three models
clf_A = DTC(random_state=1)
clf_B = SVC(random_state=1)
clf_C = BC(KNN(),random_state=1)

# TODO: Set up the training set sizes
size = [100,200,300]
clf_set = {'Decision Tree Classifier':clf_A, 'Support Vector Machine':clf_B, 'Bagging with K-Nearest Neighbors':clf_C}

# TODO: Execute the 'train_predict' function for each classifier and each training set size
print('As the instruction, I\'m using the default setting for three of the models')
print('--------------- \n')
for classifier in clf_set:
    print('Classifier: %s' %classifier)
    print(clf_set[classifier],'\n')
    for s in size:
        X = X_train[:s]
        y = y_train[:s]
        train_predict(clf_set[classifier], X, y, X_test, y_test)
    print('--------------- \n')

As the instruction, I'm using the default setting for three of the models
--------------- 

Classifier: Decision Tree Classifier
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1, splitter='best') 

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0010 seconds.
F1 score for Student dropout: 1.0000.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7143. 

Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for Student dropout: 1.0000.
Made predictions in 0.0010 seconds.
F1 sco

C:\Users\Balakdas Maharaj\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Balakdas Maharaj\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\Balakdas Maharaj\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Made predictions in 0.0551 seconds.
F1 score for Student dropout: 0.8525.
Made predictions in 0.0289 seconds.
F1 score for test set: 0.7971. 

--------------- 



** Classifer 1 - Decision Tree**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |            0.0003       |      0.0004          |         1         |       0.6942          |
| 200               |        0.0002          |            0.0002            |   1               |   0.7132              |
| 300               |             0.0003|           0.0004             |         1         |    0.7167      |

** Classifer 2 - SVM**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0010               |       0.0007       |        0.8591          |      0.7838           |
| 200               |     0.0023             |            0.0013     |          0.8693        |     0.7755            |
| 300               |          0.0061               |      0.0020                  |      0.8692            |     0.7586     |

** Classifer 3 - Gaussian Naive Bayes**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0005            |        0.0003          |      0.8550            |     0.7481            |
| 200               |       0.0005              |   0.0003             |      0.8321            |     0.7132            |
| 300               |       0.0003              |  0.0002              |      0.8088            |     0.7500            |


Note: the prediction depends on the value which is chossen for random_stae. For filling out this table I used random_state=0